In [1]:
from experta import *

Facts

In [2]:
# Fact as a subclass of dict
f = Fact('x', 'y', 'z', a=1, b=2)
print(f[0])
print(f['a'])

x
1


In [3]:
# subclass Fact to express different kinds of data or extend it with your custom functionality
class Alert(Fact):
    """The alert level."""
    pass

class Status(Fact):
    """The system status."""
    pass

f1 = Alert('red')
print(f1)

f2 = Status('critical')
print(f2)

<Undeclared Fact> Alert('red')
<Undeclared Fact> Status('critical')


Rules

In [4]:
# Rules definition - LHS: patterns/ conditions under which rule will be fired, RHS: set of actions to perform when rule is fired
class MyFact(Fact):
    pass

@Rule(MyFact()) # This is the LHS
def match_with_every_myfact():
    """This rule will match with every instance of `MyFact`."""
    # This is the RHS
    pass

@Rule(Fact('animal', family='felinae'))
def match_with_cats():
    """
    Match with every `Fact` which:
    * f[0] == 'animal'
    * f['family'] == 'felinae'
    """
    print("Meow!")

In [5]:
# Use of logic operators to express complex LHS conditions
# Check experta.operator module

class User(Fact):
    pass
@Rule(
    AND(
        OR(User('admin'),
            User('root')),
        NOT(Fact('drop-privileges'))
    )
)
def the_user_has_power():
    """
    The user is a privileged one and we are not dropping privileges.
    """
    pass

DefFacts

In [6]:
# DefFacts decorator to define a set of facts
# Will be called everytime the reset method is called

@DefFacts()
def needed_data():
    yield Fact(best_color="red")
    yield Fact(best_body="medium")
    yield Fact(best_sweetness="dry")

KnowledgeEngine

In [38]:
from experta import *
class Greetings(KnowledgeEngine):
    @DefFacts()
    def _initial_action(self):
        yield Fact(action="greet")

    
    @Rule(Fact(action="greet"), 
          NOT(Fact(name=W())))
    def ask_name(self):
        self.declare(Fact(name="Name"))
    
    @Rule(Fact(action="greet"), 
          NOT(Fact(location=W())))
    def ask_location(self):
        self.declare(Fact(location="Location"))

    @Rule(Fact(action="greet"), 
          Fact(name=MATCH.name),  # Use MATCH to bind the fact's value to a variable
          Fact(location=MATCH.location))
    def greet(self, name, location):  # Access the matched values as parameters
        print(f"Hi {name}! How is the weather in {location}?")

engine = Greetings()
engine.reset() # Prepare the engine for the execution.
engine.run() # Run it!

Hi Name! How is the weather in Location?


Handling Facts

In [17]:
# declare: adds a new fact to the list of fact known by the engine

engine = KnowledgeEngine()
engine.reset()
engine.declare(Fact(score=5))
print(engine.facts)

<f-0>: InitialFact()
<f-1>: Fact(score=5)


In [19]:
# retract: removes an existing fact from the factlist
print(engine.facts)
engine.retract(1)
print(engine.facts)

<f-0>: InitialFact()
<f-1>: Fact(score=5)
<f-0>: InitialFact()


In [27]:
# modify: Retracts some fact from the factlist and declares a new one with some changes. Changes are passed as arguments
engine = KnowledgeEngine()
engine.reset()
engine.declare(Fact(color='red'))
print(f"Before:\n{engine.facts}")

engine.modify(engine.facts[1], color='yellow', blink=True)
print(f"After:\n{engine.facts}")

Before:
<f-0>: InitialFact()
<f-1>: Fact(color='red')
After:
<f-0>: InitialFact()
<f-2>: Fact(color='yellow', blink=True)


In [28]:
# duplicate: Adds a new fact to the factlist using an existing fact as a template and adding some modifications.
engine = KnowledgeEngine()
engine.reset()
engine.declare(Fact(color='red'))
print(f"Before:\n{engine.facts}")

engine.duplicate(engine.facts[1], color='yellow', blink=True)
print(f"After:\n{engine.facts}")

Before:
<f-0>: InitialFact()
<f-1>: Fact(color='red')
After:
<f-0>: InitialFact()
<f-1>: Fact(color='red')
<f-2>: Fact(color='yellow', blink=True)


Rule

In [31]:
# Salience: This value, by default 0, determines the priority of the rule in relation to the others. Rules with a higher salience will be fired before rules with a lower one
@Rule(salience=1)
def r1():
    pass
@Rule(salience=0)
def r2():
    pass

In [46]:
# Nested Matching

from experta import *

class RockPaperScissor(KnowledgeEngine):
    @DefFacts()
    def _initial_action(self):
        yield Fact(name="scissors", against={"scissors": 0, "rock": -1, "paper": 1})
        yield Fact(name="paper", against={"scissors": -1, "rock": 1, "paper": 0})
        yield Fact(name="rock", against={"scissors": 1, "rock": 0, "paper": -1})

    @Rule(Fact(name=MATCH.name, against__scissors=1, against__paper=-1))
    def what_wins_to_scissors_and_losses_to_paper(self,name):
        print(name)


engine = RockPaperScissor()
engine.reset()
engine.run()

rock


In [65]:
# Nested matching with arbitrary deep structure
class Ship(Fact):
    pass

ship_1 = Ship(data={
    "name": "SmallShip",
    "position": {
        "x": 300,
        "y": 200
    },
    "parent": {
        "name": "BigShip",
        "position": {
            "x": 300,
            "y": 200
        }
    }
})
print(ship_1)

class Ships(KnowledgeEngine):
    @DefFacts()
    def _initial_action(self):
        yield ship_1

    @Rule(Ship(data__name=MATCH.name1,
            data__position__x=MATCH.x1,
            data__position__y=MATCH.y1,
            data__parent__name=MATCH.name2,
            data__parent__position__x=MATCH.x2,
            data__parent__position__y=MATCH.y2),
            TEST(lambda x1,x2: x1==x2),
            TEST(lambda y1,y2: y1==y2))   
    def collision_detected(self, name1, name2, **_):
        print("COLLISION!", name1, name2)

engine = Ships()
engine.reset()
engine.run()

<Undeclared Fact> Ship(data=<frozendict {'name': 'SmallShip', 'position': <frozendict {'x': 300, 'y': 200}>, 'parent': <frozendict {'name': 'BigShip', 'position': <frozendict {'x': 300, 'y': 200}>}>}>)
COLLISION! SmallShip BigShip
